In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization, LeakyReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
%matplotlib inline

Using TensorFlow backend.


In [4]:
import sys
p = sorted(sys.path)
for el in p:
    print(el)


/home/ec2-user/.ipython
/home/ec2-user/.local/lib/python3.4/site-packages
/home/ec2-user/src/caffe_python3/python
/home/ec2-user/src/cntk/bindings/python
/usr/lib/python3.4/dist-packages
/usr/lib/python3.4/dist-packages/IPython/extensions
/usr/lib/python3.4/dist-packages/Keras-2.0.8-py3.4.egg
/usr/lib/python3.4/dist-packages/certifi-2017.7.27.1-py3.4.egg
/usr/lib/python3.4/dist-packages/chardet-3.0.4-py3.4.egg
/usr/lib/python3.4/dist-packages/idna-2.6-py3.4.egg
/usr/lib/python3.4/dist-packages/mxnet-0.11.0-py3.4.egg
/usr/lib/python3.4/dist-packages/protobuf-3.0.0b2-py3.4.egg
/usr/lib/python3.4/dist-packages/requests-2.18.4-py3.4.egg
/usr/lib/python3.4/dist-packages/urllib3-1.22-py3.4.egg
/usr/lib64/python3.4
/usr/lib64/python3.4/dist-packages
/usr/lib64/python3.4/lib-dynload
/usr/lib64/python3.4/plat-linux
/usr/local/lib/python3.4/site-packages
/usr/local/lib/python3.4/site-packages/Theano-0.9.0-py3.4.egg
/usr/local/lib64/python3.4/site-packages


In [2]:
data_train = pd.read_csv("data_train.csv")

In [3]:
data_train.head()

,id,molecule_name,atom_index_0,atom_index_1,scalar_coupling_constant,x_0,y_0,z_0,x_1,y_1,...,type_1JHN,type_2JHC,type_2JHH,type_2JHN,type_3JHC,type_3JHH,type_3JHN,atom_1_C,atom_1_H,atom_1_N
0,0,dsgdb9nsd_000001,1,0,84.8076,0.002150,-0.006031,0.001976,-0.012698,1.085804,...,0,0,0,0,0,0,0,1,0,0
1,1,dsgdb9nsd_000001,1,2,-11.2570,0.002150,-0.006031,0.001976,1.011731,1.463751,...,0,0,1,0,0,0,0,0,1,0
2,2,dsgdb9nsd_000001,1,3,-11.2548,0.002150,-0.006031,0.001976,-0.540815,1.447527,...,0,0,1,0,0,0,0,0,1,0
3,3,dsgdb9nsd_000001,1,4,-11.2543,0.002150,-0.006031,0.001976,-0.523814,1.437933,...,0,0,1,0,0,0,0,0,1,0
4,4,dsgdb9nsd_000001,2,0,84.8074,1.011731,1.463751,0.000277,-0.012698,1.085804,...,0,0,0,0,0,0,0,1,0,0


In [4]:
data_test = pd.read_csv("data_test.csv")
data_test.head()

,id,molecule_name,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,...,type_1JHN,type_2JHC,type_2JHH,type_2JHN,type_3JHC,type_3JHH,type_3JHN,atom_1_C,atom_1_H,atom_1_N
0,4658147,dsgdb9nsd_000004,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,...,0,1,0,0,0,0,0,1,0,0
1,4658148,dsgdb9nsd_000004,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,...,0,0,0,0,0,0,0,1,0,0
2,4658149,dsgdb9nsd_000004,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,...,0,0,0,0,0,1,0,0,1,0
3,4658150,dsgdb9nsd_000004,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,...,0,0,0,0,0,0,0,1,0,0
4,4658151,dsgdb9nsd_000004,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,...,0,1,0,0,0,0,0,1,0,0


In [5]:
print(data_test.shape)

(2505542, 26)


In [6]:
print(data_train.shape)

(4658147, 55)


In [7]:
from keras import backend as K
from keras.losses import mean_absolute_error
import tensorflow as tf

In [8]:
print(tf.__version__)

1.3.0


In [9]:
def logloss(y_true, y_pred):
    return tf.log(mean_absolute_error(y_true,y_pred))

In [10]:
features = set(data_train.columns).intersection(set(data_test.columns))
targets = set(data_train.columns).difference(set(data_test.columns))
features.remove('id')
features.remove('molecule_name')
features.remove('atom_index_1')
features.remove('atom_index_0')
print(features, len(features), len(data_test.columns))
print(targets, len(targets))

{'z_1', 'atom_1_H', 'atom_1_C', 'y_diff', 'y_1', 'dist', 'x_1', 'type_2JHH', 'type_1JHN', 'type_3JHC', 'y_0', 'z_diff', 'type_1JHC', 'type_2JHN', 'x_0', 'type_3JHH', 'z_0', 'type_2JHC', 'num_atoms', 'x_diff', 'type_3JHN', 'atom_1_N'} 22 26
{'ZZ_1', 'YY_1', 'fc', 'scalar_coupling_constant', 'YX_0', 'ZY_0', 'dipole_moment_y', 'ZX_0', 'pso', 'mulliken_charge_0', 'potential_energy', 'sd', 'ZX_1', 'YX_1', 'mulliken_charge_1', 'dipole_moment_z', 'YZ_1', 'XY_0', 'XX_1', 'YY_0', 'dso', 'YZ_0', 'XX_0', 'XZ_1', 'XZ_0', 'dipole_moment_x', 'ZZ_0', 'ZY_1', 'XY_1'} 29


In [11]:
target_attr = targets.copy()
target_attr.remove('scalar_coupling_constant')

In [12]:
data_features_train = data_train[list(features)]
data_targets_train = data_train[list(target_attr)]
data_features_test = data_test[list(features)]

In [13]:
all_inputs = pd.concat([data_features_train, data_features_test])
input_data = StandardScaler().fit_transform(all_inputs)
input_data = pd.DataFrame(input_data, index=all_inputs.index, columns=all_inputs.columns)

In [14]:
data_test.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'x_0', 'y_0',
       'z_0', 'x_1', 'y_1', 'z_1', 'x_diff', 'y_diff', 'z_diff', 'dist',
       'num_atoms', 'type_1JHC', 'type_1JHN', 'type_2JHC', 'type_2JHH',
       'type_2JHN', 'type_3JHC', 'type_3JHH', 'type_3JHN', 'atom_1_C',
       'atom_1_H', 'atom_1_N'],
      dtype='object')

In [15]:
output_data = StandardScaler().fit_transform(data_targets_train)
output_data = pd.DataFrame(output_data, index=data_targets_train.index, columns=data_targets_train.columns)
output_data['scalar_coupling_constant'] = data_train["scalar_coupling_constant"]

In [16]:
input_data.head()

,z_1,atom_1_H,atom_1_C,y_diff,y_1,dist,x_1,type_2JHH,type_1JHN,type_3JHC,...,type_1JHC,type_2JHN,x_0,type_3JHH,z_0,type_2JHC,num_atoms,x_diff,type_3JHN,atom_1_N
0,-0.057304,-0.512289,0.621805,0.851407,0.840693,-1.722430,-0.075781,-0.297134,-0.097573,-0.692604,...,2.360517,-0.162218,-0.060235,-0.380994,-0.030222,-0.569374,-5.054915,-0.004313,-0.192912,-0.276349
1,-0.063452,1.952025,-1.608221,1.105402,1.055052,-0.784250,0.618133,3.365485,-0.097573,-0.692604,...,-0.423636,-0.162218,-0.060235,-0.380994,-0.030222,-0.569374,-5.054915,0.726043,-0.192912,-0.276349
2,-0.761370,1.952025,-1.608221,1.094499,1.045850,-0.784212,-0.433510,3.365485,-0.097573,-0.692604,...,-0.423636,-0.162218,-0.060235,-0.380994,-0.030222,-0.569374,-5.054915,-0.380829,-0.192912,-0.276349
3,0.657706,1.952025,-1.608221,1.088051,1.040409,-0.784199,-0.421993,3.365485,-0.097573,-0.692604,...,-0.423636,-0.162218,-0.060235,-0.380994,-0.030222,-0.569374,-5.054915,-0.368708,-0.192912,-0.276349
4,-0.057304,-0.512289,0.621805,-0.136346,0.840693,-1.722432,-0.075781,-0.297134,-0.097573,-0.692604,...,2.360517,-0.162218,0.514406,-0.380994,-0.031308,-0.569374,-5.054915,-0.724084,-0.192912,-0.276349


In [17]:
output_data.head()

,dipole_moment_z,YZ_1,XY_0,XX_1,ZZ_1,YY_0,dso,YY_1,fc,YX_0,...,dipole_moment_x,ZZ_0,potential_energy,sd,ZY_1,ZX_1,YX_1,XY_1,mulliken_charge_1,scalar_coupling_constant
0,-0.224683,-0.016132,-0.538167,1.104354,1.448631,-0.05429,0.543911,1.287560,1.956281,-0.510046,...,0.008861,1.449250,9.78763,1.242269,-0.014925,-0.002748,0.002531,0.003042,-2.057703,84.8076
1,-0.224683,-0.106971,-0.538167,-0.972567,-1.326758,-0.05429,-3.442315,-1.034857,-0.776777,-0.510046,...,0.008861,1.449250,9.78763,1.951955,-0.107775,-0.221518,0.051591,0.052808,0.729230,-11.2570
2,-0.224683,0.079809,-0.538167,-0.973381,-1.411674,-0.05429,-3.442606,-0.965034,-0.776713,-0.510046,...,0.008861,1.449250,9.78763,1.951710,0.083135,0.064371,0.168119,0.171020,0.729234,-11.2548
3,-0.224683,0.075897,-0.538167,-0.974831,-1.412647,-0.05429,-3.442670,-0.962650,-0.776695,-0.510046,...,0.008861,1.449250,9.78763,1.951637,0.079135,-0.064951,-0.162477,-0.164347,0.729234,-11.2543
4,-0.224683,-0.016132,0.521036,1.104354,1.448631,-0.06957,0.543914,1.287560,1.956275,0.491207,...,0.008861,1.399787,9.78763,1.242313,-0.014925,-0.002748,0.002531,0.003042,-2.057703,84.8074


Callbacks

In [18]:
output_data.columns

Index(['dipole_moment_z', 'YZ_1', 'XY_0', 'XX_1', 'ZZ_1', 'YY_0', 'dso',
       'YY_1', 'fc', 'YX_0', 'YZ_0', 'ZY_0', 'XX_0', 'dipole_moment_y', 'ZX_0',
       'pso', 'XZ_1', 'mulliken_charge_0', 'XZ_0', 'dipole_moment_x', 'ZZ_0',
       'potential_energy', 'sd', 'ZY_1', 'ZX_1', 'YX_1', 'XY_1',
       'mulliken_charge_1', 'scalar_coupling_constant'],
      dtype='object')

In [ ]:
# Callback for Early Stopping... May want to raise the min_delta for small numbers of epochs
es = EarlyStopping(monitor='val_loss', min_delta=0.0005, patience=8,verbose=1, mode='auto')
# Callback for Reducing the Learning Rate... when the monitor levels out for 'patience' epochs, then the LR is reduced
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=7, min_lr=1e-6, mode='auto', verbose=1)
# Save the best value of the model for future use
sv_mod = ModelCheckpoint("best_model.hdf5", monitor='val_loss', save_best_only=True, period=1)

In [ ]:
vars_in = Input(shape=(len(features),))
x = Dense(256)(vars_in)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.05)(x)
x = Dropout(0.2)(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.05)(x)
x = Dropout(0.2)(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.05)(x)
x = Dropout(0.2)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.05)(x)
x = Dropout(0.2)(x)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.05)(x)
x = Dropout(0.2)(x)
vars_out = Dense(len(targets), activation="linear")(x)
model = Model(inputs=vars_in, outputs=vars_out)
model.summary()
model.compile(loss="mae", optimizer='adam')
history = model.fit(np.array(input_data.iloc[:len(data_train),:]), np.array(output_data), validation_split=0.2, nb_epoch=200, shuffle=True, batch_size=1024)
model.save('model.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 22)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               5888      
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              263168    
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
__________

/usr/lib/python3.4/dist-packages/ipykernel_launcher.py:26: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 3726517 samples, validate on 931630 samples
Epoch 1/200
3726517/3726517 [==============================] - 101s - loss: 0.7217 - val_loss: 0.6347
Epoch 2/200
3726517/3726517 [==============================] - 101s - loss: 0.6757 - val_loss: 0.6173
Epoch 3/200
1215488/3726517 [========>.....................] - ETA: 63s - loss: 0.6685

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
input_data.iloc[:len(data_train),:].head()

In [ ]:
answers_train = model.predict(np.array(input_data.iloc[:len(data_train),:]), batch_size=1024)

In [ ]:
answers = model.predict(np.array(input_data.iloc[len(data_train):,:]), batch_size=1024)

In [ ]:


print(np.log(np.mean(np.abs(answers_train[:,28]-data_train['scalar_coupling_constant'].values))))

In [ ]:
data_test["answer"] = answers[:,28]
data_test[['id','answer']].to_csv("submission_new.csv", index=False, header=['id', 'scalar_coupling_constant'])